In [ ]:
import os
import logging
import numpy as np
import bpy
from scipy.spatial.transform import Rotation as R

import tensorflow as tf
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import json
import urllib3
import shutil
import tarfile
import sys
import os
os.environ["KUBRIC_USE_GPU"] = "1"

http = urllib3.PoolManager()

import kubric as kb
from kubric.renderer import Blender

In [ ]:
devnull = os.open('/dev/null', os.O_WRONLY)
os.dup2(devnull, 1)
os.close(devnull)

In [ ]:
shapenet_gs = 'gs://kubric-unlisted/assets/ShapeNetCore.v2/'
manifest_gs = 'gs://kubric-unlisted/assets/ShapeNetCore.v2.json'
manifest_path = Path('ShapeNetCore.v2.json')
shapenet_path = Path('shapenet')
renders_path = Path('renders')

In [ ]:
def fetch_gs(src_gs, dest_path):
    if not dest_path.exists():
        dest_path.parent.mkdir(parents=True, exist_ok=True)
        tf.io.gfile.copy(src_gs, dest_path)

def fetch_url(src_url, dest_path):
    if not dest_path.exists():
        dest_path.parent.mkdir(parents=True, exist_ok=True)
        with http.request('GET', src_url, preload_content=False) as r, open(dest_path, 'wb') as dest_file:
            shutil.copyfileobj(r, dest_file)

In [ ]:
class ShapeNet():
    def __init__(self):
        fetch_gs(manifest_gs, manifest_path)
        with open(manifest_path, 'rb') as manifest_file:
            manifest = json.loads(manifest_file.read())
        self.assets = manifest['assets']
    
    def fetch_asset_tar(self, asset_id):
        path = shapenet_path / (asset_id + ".tar.gz")
        fetch_gs(shapenet_gs + (asset_id + ".tar.gz"), path)
        return path
    
    def fetch_asset(self, asset_id):
        path = shapenet_path / asset_id
        if not path.is_dir():
            tar_path = self.fetch_asset_tar(asset_id)
            with tarfile.open(tar_path) as tar:
                tar.extractall(path)
        return path
    
    def get_category(self, category):
        return [asset_id for asset_id, asset in self.assets.items() if asset['metadata']['category'] == category]
    
    def fetch_category_tar(self, category):
        asset_ids = self.get_category(category)
        for i in tqdm(range(len(asset_ids)), desc='Fetching tar...'):
            asset_id = asset_ids[i]
            self.fetch_asset_tar(asset_id)
    
    def fetch_category(self, category):
        asset_ids = self.get_category(category)
        for i in tqdm(range(len(asset_ids)), desc='Fetching...'):
            asset_id = asset_ids[i]
            self.fetch_asset(asset_id)
    
    def create(self, asset_id):
        asset_entry = self.assets[asset_id]
        asset_type = kb.AssetSource._resolve_asset_type(asset_entry["asset_type"])
        asset_dir = self.fetch_asset(asset_id)

        asset_kwargs = asset_entry.get("kwargs", {})
        asset_kwargs = kb.AssetSource._adjust_paths(asset_kwargs, asset_dir)
        if asset_type == kb.core.FileBasedObject:
            asset_kwargs["asset_id"] = asset_id
        asset = asset_type(**asset_kwargs)
        asset.metadata.update(asset_entry.get("metadata", {}))

        return asset

In [ ]:
def setup(camera_angles, resolution):
    scene = kb.Scene(resolution=resolution)
    renderer = Blender(scene=scene, use_denoising=True, adaptive_sampling=False)

    scene += kb.Cube(name='floor', scale=(2, 2, 0.1), position=(0, 0, -0.1))
    scene += kb.Cube(name='nwall', scale=(0.1, 2, 1), position=(-2, 0, 0.9))
    scene += kb.Cube(name='ewall', scale=(2, 0.1, 1), position=(0, 2, 0.9))
    scene += kb.Cube(name='swall', scale=(0.1, 2, 1), position=(2, 0, 0.9))
    scene += kb.Cube(name='wwall', scale=(2, 0.1, 1), position=(0, -2, 0.9))
    scene += kb.Cube(name='ceiling', scale=(2, 2, 0.1), position=(0, 0, 1.9))

    light_pose = R.from_euler('xz', (50, 70), degrees=True).apply([0, 2, 0])
    scene += kb.PointLight(name="point_light", position=light_pose, intensity=15)

    scene.camera = kb.PerspectiveCamera()
    centroid = (0, 0, 0.2)
    camera_poses = [centroid + R.from_euler('xz', (15, a), degrees=True).apply([0, 1.5, 0]) for a in camera_angles]
    scene.frame_end = len(camera_poses)
    for i in range(scene.frame_start, scene.frame_end + 1):
        scene.camera.position = camera_poses[i-1]
        scene.camera.look_at(centroid)
        scene.camera.keyframe_insert("position", i)
        scene.camera.keyframe_insert("quaternion", i)
    
    return scene, renderer

In [ ]:
def center_obj(obj):
    obj.quaternion = kb.Quaternion(axis=[1, 0, 0], degrees=90)
    obj.position = (0, 0, obj.position[2] - obj.aabbox[0][2])

In [ ]:
def save_frame(frame, asset_id, names):
    asset_render_path = renders_path / asset_id
    asset_render_path.mkdir(parents=True, exist_ok=True)
    
    for i in range(len(frame['rgba'])):
        color = frame['rgba'][i]
        color_image = Image.fromarray(color)
        filename = 'color_' + names[i] + '.png'
        color_image.save(asset_render_path / filename)
        
    for i in range(len(frame['depth'])):
        depth = frame['depth'][i]
        depth = (65535 * (1 - depth / 5))
        depth_image = Image.fromarray(depth.reshape(depth.shape[:2]).astype(np.uint16))
        filename = 'depth_' + names[i] + '.png'
        depth_image.save(asset_render_path / filename)

In [ ]:
def render_batch(camera_angles, resolution, asset_ids):
    scene, renderer = setup([], resolution)
    names = [f'angle{camera_angle}_res{resolution[0]}x{resolution[1]}' for camera_angle in camera_angles]
    objs = [shapenet.create(asset_id) for asset_id in asset_ids]
    
    for i in tqdm(range(len(asset_ids)), desc='Rendering...'):
        asset_id = asset_ids[i]
        obj = objs[i]
        center_obj(obj)
        scene.add(obj)
        frame = renderer.render(return_layers=("rgba", 'depth'))
        scene.remove(obj)
        save_frame(frame, asset_id, names)

In [ ]:
def render_batch_random_angles(resolution, asset_ids):
    scene, renderer = setup([], resolution)
    objs = [shapenet.create(asset_id) for asset_id in asset_ids]
    centroid = (0, 0, 0.2)
    
    for i in tqdm(range(len(asset_ids)), desc='Rendering...'):
        asset_id = asset_ids[i]
        obj = objs[i]
        center_obj(obj)
        scene.add(obj)
        angle = np.random.random() * 360
        scene.camera.position = centroid + R.from_euler('xz', (15, angle), degrees=True).apply([0, 1.5, 0])
        scene.camera.look_at(centroid)
        name = f'random_angle{int(angle)}_res{resolution[0]}x{resolution[1]}'
        frame = renderer.render_still(return_layers=("rgba", 'depth'))
        scene.remove(obj)
        save_frame({k: [v] for k, v in frame.items()}, asset_id, [name])

In [11]:
shapenet = ShapeNet()
cars = shapenet.get_category('car')

In [21]:
resolution = (512, 512)
batch_size = 100
batch_indices = list(range(100, len(cars), batch_size))
for start in batch_indices:
    print(f'Rendering batch {start} to {min(start+batch_size, len(cars))}')
    render_batch_random_angles(resolution, cars[start:start+batch_size])

Rendering batch 100 to 200


Rendering...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:06<00:00,  4.27s/it]

Rendering batch 200 to 300



Rendering...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:10<00:00,  4.30s/it]

Rendering batch 300 to 400


...


Rendering...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:03<00:00,  4.24s/it]

Rendering batch 3300 to 3400



Rendering...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:06<00:00,  4.27s/it]

Rendering batch 3400 to 3486



Rendering...: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [06:05<00:00,  4.25s/it]


In [ ]:
camera_angles = [30, 60, 150]
resolution = (256, 256)

In [ ]:
batch_size = 100
batch_indices = list(range(3400, len(cars), batch_size))
for start in batch_indices:
    print(f'Rendering batch {start} to {min(start+batch_size, len(cars))}')
    render_batch(camera_angles, resolution, cars[start:start+batch_size])

In [ ]:
render_batch([40], (1024, 1024), [cars[4]])